In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen

In [ ]:
import json
import sys
sys.setrecursionlimit(1500)

In [ ]:
def parse_return_lists(url):
    lists = list()
    try:
        r = requests.get(url)
        if r.status_code == 200:
            html = urlopen(url) 
            soup = BeautifulSoup(html, 'html.parser')
            
            #find all lists on the given page
            elems = soup.find_all("li")
            
            
            for x in elems:
                cite = x.select('a[href]')
                if cite!=[]:
                    
                    #extract links to lists
                    for x in cite:
                        if (x.attrs['href'].startswith("/wiki/Lists") or "lists" in x.attrs['href']) and x.attrs['href'] not in listoflists:
                            yield x.attrs['href']
                        elif x.attrs['href'].startswith("/wiki/List_of"):
                            if cite[0].attrs['href'].startswith("/wiki/List_of_lists_of_lists")!=True:
                                yield x.attrs['href']
    except:
        print(url)
        pass

In [ ]:
#parse list of lists of lists, extract links to the lists

url = 'https://en.wikipedia.org/wiki/List_of_lists_of_lists'
urls = []
l = parse_return_lists(url)

In [ ]:
listoflists  =[]
for x in l:
    if x not in listoflists:
        listoflists.append(x)
print(len(listoflists))

In [ ]:
#saves "List of", that doesn't need further parsing, to lists; returns "Lists of", that need further parsing
def extract_lists(t):
    t2 = []
    for x in t:
        if x.startswith("/wiki/List_of") and x not in lists:
            lists.append(x)
        else:
            t2.append(x)
    print("Parsed: ",len(lists))
    print("To parse: ", len(t2))
    return t2

In [ ]:
#returns "List of" and "Lists of", mixed
def parse_it(it):
    temp = []
    for x in it:
        l = parse_return_lists("https://en.wikipedia.org"+x)
        for y in l:
            if y not in temp:
                temp.append(y)

    print("New: ",len(temp))
    return temp

In [ ]:
lists = []
it1 = extract_lists(listoflists)

In [ ]:
def extraction(i, current_it):
    if i<20:
        print(i)
        next_it = parse_it(current_it)
        next_it = extract_lists(next_it)
        df = pd.DataFrame({"list":lists})
        df.to_csv("wikilists_depth_"+str(i)+".tsv", sep = '\t')
        df = pd.DataFrame({"list":it3})
        df.to_csv("wikilists_to_parse_depth_"+str(i)+".tsv", sep = '\t')
        i+=1
        extraction(i, current_it)
    else:
        print("Complete WikiLists parsing to depth 20")

In [ ]:
i = 1
extraction(i, it1)